In [26]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [40]:
df_producto=pd.read_csv('../data/IMPO_ITEM_12019000.csv', encoding='latin-1',sep=';', dtype={'CUIT':'str',
                                                                                            'dia':'str',
                                                                                            'mes':'str',
                                                                                            'anio':'str'})
df_producto.pais_descri=df_producto.pais_descri.apply(lambda x: x.capitalize())
df_producto.empresa=df_producto.empresa.apply(lambda x: x.lower())
df_producto['fob_unitario_ton']=df_producto.fob/df_producto.pnet*1000
bm=pd.read_excel('../data/CMO-Historical-Data-Monthly.xlsx',sheet_name='Monthly Prices',skiprows=6)
bm=bm[[bm.columns[0],bm.columns[24]]].rename(columns={'Unnamed: 0':'Fecha', 'SOYBEANS':'soja_bm'})
bm['anio']=bm.Fecha.apply(lambda x: x[:4])
bm['mes']=bm.Fecha.apply(lambda x: x[5:])
bm.drop('Fecha',axis=1,inplace=True)

df_producto=pd.merge(left=df_producto, right=bm, how='left', on=['anio','mes'])
df_producto['diferencia_ref']=df_producto.fob_unitario_ton-df_producto.soja_bm

#Capitalizo los precios al valor del ultimo mes para que se puedan comparar.
df_producto['fob_unitario_ton_capit']=df_producto.fob_unitario_ton*df_producto.soja_bm.iloc[-1]/df_producto.soja_bm
df_producto['diferencia_ref_capi']=df_producto.fob_unitario_ton_capit-df_producto.soja_bm.iloc[-1]
df_producto['fecha']=df_producto["mes"].astype(str)+'-'+df_producto['anio'].astype(str)
df_producto['fecha']=pd.to_datetime(df_producto["fecha"]).dt.strftime('%m-%Y')
df_producto=df_producto.sort_values(['anio','mes','dia'], ascending=True)

desde=df_producto['anio'][0]
hasta=df_producto['anio'].iloc[-1]

In [28]:
#No hay outliers
df_producto.sort_values('fob_unitario_ton',ascending=True)

,dia,mes,anio,CUIT,empresa,CNRO_ENMIENDA,NOMEN,nomen_descri,pais,pais_descri,cif,fob,pnet,fob_unitario_ton,soja_bm,diferencia_ref,fob_unitario_ton_capit,diferencia_ref_capi,fecha
414,29,04,2021,30578036071,mirgor sa c i f i a,6,12019000,Porotos de soja excluidos p/siembra,200,Argentina,290152.77,270471.78,660500,409.495503,597.13,-187.634497,378.512289,-173.437711,04-2021
76,06,05,2021,33716902299,alfatrans s.a.s. s. a. s.,6,12019000,Porotos de soja excluidos p/siembra,200,Argentina,14611.90,12306.60,28620,430.000000,646.80,-216.800000,366.942641,-185.007359,05-2021
326,22,12,2021,30700869918,bunge argentina s a,6,12019000,Porotos de soja excluidos p/siembra,221,Paraguay,761648.72,727323.27,1674763,434.284296,554.14,-119.855704,432.567974,-119.382026,12-2021
306,21,12,2021,30700869918,bunge argentina s a,6,12019000,Porotos de soja excluidos p/siembra,221,Paraguay,837191.31,799461.37,1840871,434.284298,554.14,-119.855702,432.567976,-119.382024,12-2021
373,26,06,2021,33502232229,viterra argentina s.a.,6,12019000,Porotos de soja excluidos p/siembra,202,Bolivia,2194935.89,1972350.84,4416370,446.600000,614.68,-168.080000,401.023085,-150.926915,06-2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,15,03,2022,30715118773,molinos agro s.a.,6,12019000,Porotos de soja excluidos p/siembra,221,Paraguay,7938323.26,7679626.00,11228180,683.960001,720.60,-36.639999,523.885266,-28.064734,03-2022
390,28,03,2022,33506737449,cofco international argentina s.a.,6,12019000,Porotos de soja excluidos p/siembra,221,Paraguay,17603773.99,17055990.35,24899256,685.000000,720.60,-35.600000,524.681862,-27.268138,03-2022
369,26,03,2022,33506737449,cofco international argentina s.a.,6,12019000,Porotos de soja excluidos p/siembra,221,Paraguay,4127713.45,3999269.75,5838350,685.000000,720.60,-35.600000,524.681862,-27.268138,03-2022
214,15,03,2022,33502232229,viterra argentina s.a.,6,12019000,Porotos de soja excluidos p/siembra,221,Paraguay,11725498.90,11343474.58,16445300,689.770000,720.60,-30.830000,528.335486,-23.614514,03-2022


In [49]:
def plot_precio_soja(y=['fob_unitario_ton', 'diferencia_ref'], color=['pais_descri','empresa']):
     if y=='fob_unitario_ton':
          title_1= 'Fob unitario'
          title_2= 'FOB por tonelada importada de soja <br> <sup> Precio de referencia del Banco Mundial'
     elif y=='diferencia_ref':
          title_1= 'Diferencia en USD'
          title_2= 'Diferencia con precio de referencia en USD por tonelada importada de soja <br> <sup> Precio de referencia del Banco Mundial'
     precio_soja_plot=px.scatter(
     df_producto.round(1),
     x='fecha',
     y=y,
     color=color,
     labels={
          "pais_descri": "Destino",
          'fob_unitario_ton': 'Precio',
          'fob':'Fob',
          'pnet':'Kg',
          'fecha':'Fecha',
          'soja_bm': 'Precio de referencia',
          'diferencia_ref':'Spread',
          'empresa':'Importador'
          },
     hover_data={'pais_descri',
                    'fob_unitario_ton',
                    'soja_bm',
                    'diferencia_ref',
                    'empresa',
                    'fob',
                    'pnet'        
                                   }
     )

     precio_soja_plot.update_yaxes(title_text= title_1, 
                                   range=[df_producto[y].min()-100,df_producto[y].max()+100]
                                   )
     precio_soja_plot.update_xaxes(title_text='')

     precio_soja_plot.update_traces(marker=dict(size=12,
                                   line=dict(width=2,
                                             color='DarkSlateGrey')),
                    selector=dict(mode='markers'))

     precio_soja_plot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=700, width=1000,
                                   template = 'none',
                                   title_text=title_2)

     return precio_soja_plot

def precio_soja_boxplot_mes():
     precio_soja_boxplot=px.box(
     df_producto.round(1),
     x='fecha',
     y='fob_unitario_ton',
     color='fecha',
     labels={
          "pais_descri": "Destino",
          'fob_unitario_ton': 'Precio',
          'fob':'Fob',
          'pnet':'Kg',
          'fecha':'Fecha',
          'soja_bm': 'Precio de referencia',
          'diferencia_ref':'Spread',
          'empresa':'Importador'
          },
     hover_data={'pais_descri',
                    'fob_unitario_ton',
                    'soja_bm',
                    'diferencia_ref',
                    'empresa',
                    'fob',
                    'pnet'        
                                   }
     )

     precio_soja_boxplot.update_yaxes(title_text= 'Fob unitario', 
                                   range=[df_producto['fob_unitario_ton'].min()-100,df_producto['fob_unitario_ton'].max()+100]
                                   )
     precio_soja_boxplot.update_xaxes(title_text='')

     precio_soja_boxplot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=700, width=1000,
                                   template = 'none',
                                   title_text='FOB por tonelada importada de soja <br> <sup> Precio de referencia del Banco Mundial',
                                   showlegend=False)

     return precio_soja_boxplot

def precio_soja_boxplot_capitalizado():
     precio_soja_boxplot=px.box(
     df_producto.round(1),
     # x='fecha',
     x='fob_unitario_ton_capit',
     # color='fecha',
     labels={
          "pais_descri": "Destino",
          'fob_unitario_ton': 'Precio',
          'fob_unitario_ton_capit':'Precio capitalizado',
          'fob':'Fob',
          'pnet':'Kg',
          'fecha':'Fecha',
          'soja_bm': 'Precio de referencia',
          'diferencia_ref':'Spread',
          'empresa':'Importador'
          },
     hover_data={'pais_descri',
                    'fob_unitario_ton',
                    'fob_unitario_ton_capit',
                    'soja_bm',
                    'diferencia_ref',
                    'empresa',
                    'fob',
                    'pnet'        
                                   }
     )

     precio_soja_boxplot.update_yaxes(title_text= '', 
                                   # range=[df_producto['fob_unitario_ton_capit'].min()-100,df_producto['fob_unitario_ton_capit'].max()+100]
                                   )
     precio_soja_boxplot.update_xaxes(title_text='Fob Unitario')

     precio_soja_boxplot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=400, width=700,
                                   template = 'none',
                                   title_text='FOB por tonelada importada de soja <br> <sup> Capitalizado al último precio disponible. Precio de referencia del Banco Mundial',
                                   showlegend=False)

     return precio_soja_boxplot

def precio_soja_boxplot():
     precio_soja_boxplot=px.box(
     df_producto.round(1),
     # x='fecha',
     y='fob_unitario_ton',
     # color='fecha',
     labels={
          "pais_descri": "Destino",
          'fob_unitario_ton': 'Precio',
          'fob':'Fob',
          'pnet':'Kg',
          'fecha':'Fecha',
          'soja_bm': 'Precio de referencia',
          'diferencia_ref':'Spread',
          'empresa':'Importador'
          },
     hover_data={'pais_descri',
                    'fob_unitario_ton',
                    'soja_bm',
                    'diferencia_ref',
                    'empresa',
                    'fob',
                    'pnet'        
                                   }
     )

     precio_soja_boxplot.update_yaxes(title_text= 'Fob unitario', 
                                   range=[df_producto['fob_unitario_ton'].min()-100,df_producto['fob_unitario_ton'].max()+100]
                                   )
     precio_soja_boxplot.update_xaxes(title_text='')

     precio_soja_boxplot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=700, width=500,
                                   template = 'none',
                                   title_text='FOB por tonelada importada de soja <br> <sup> Precio de referencia del Banco Mundial',
                                   showlegend=False)

     return precio_soja_boxplot
precio_soja_boxplot_capitalizado()
# Acá hay algo para ver, no detecta a las expo paraguayas como outliers... Sesgo a mostrar a los valores superiores.
# plot_precio_soja('diferencia_ref', 'empresa')

In [30]:
plot_precio_soja(y='diferencia_ref',color='pais_descri').write_html('../output/Impo diferencia con ref soja pais.html')
plot_precio_soja(y='fob_unitario_ton',color='pais_descri').write_html('../output/Impo fob unitario por ton soja pais.html')
plot_precio_soja(y='diferencia_ref',color='empresa').write_html('../output/Impo diferencia con ref soja empresa.html')
plot_precio_soja(y='fob_unitario_ton',color='empresa').write_html('../output/Impo fob unitario por ton soja empresa.html')
precio_soja_boxplot().write_html('../output//Impo boxplot sin capitalizar.html')
precio_soja_boxplot_capitalizado().write_html('../output//Impo boxplot capitalizado.html')

## Empresas importadoras Frecuencia Mensual

In [31]:
gb = df_producto.groupby(['anio','mes','NOMEN','nomen_descri'])
counts = gb.size().to_frame(name='n_operaciones')
df_mensual=(counts
 .join(gb.agg({'CUIT': 'nunique'}).rename(columns={'CUIT': 'n_empresas'}))
 .join(gb.agg({'fob': 'sum'}).rename(columns={'fob': 'fob'}))
 .join(gb.agg({'pnet': 'sum'}).rename(columns={'pnet': 'pnet'}))
 .reset_index()
)

df_mensual['fob_unitario_ton']=df_mensual.fob/df_mensual.pnet*1000

df_mensual_refer=pd.merge(left=df_mensual,right=bm,how='left', on=['anio','mes'])
df_mensual_refer['diferencia_ref']=df_mensual_refer.fob_unitario_ton-df_mensual_refer.soja_bm
df_mensual_refer['fob_unitario_ton_capit']=df_mensual_refer.fob_unitario_ton*df_mensual_refer.soja_bm.iloc[-1]/df_mensual_refer.soja_bm
df_mensual_refer['diferencia_ref_capi']=df_mensual_refer.fob_unitario_ton_capit-df_mensual_refer.soja_bm.iloc[-1]

df_mensual_refer

,anio,mes,NOMEN,nomen_descri,n_operaciones,n_empresas,fob,pnet,fob_unitario_ton,soja_bm,diferencia_ref,fob_unitario_ton_capit,diferencia_ref_capi
0,2021,01,12019000,Porotos de soja excluidos p/siembra,7,2,3.014608e+07,59320796,508.187339,576.37,-68.182661,597.971187,-80.228813
1,2021,02,12019000,Porotos de soja excluidos p/siembra,2,1,1.194239e+07,22117986,539.940000,578.26,-38.320000,633.257199,-44.942801
2,2021,03,12019000,Porotos de soja excluidos p/siembra,42,6,4.496596e+08,885870939,507.590364,585.71,-78.119636,587.744421,-90.455579
3,2021,04,12019000,Porotos de soja excluidos p/siembra,38,7,3.632979e+08,719101798,505.210686,597.13,-91.919314,573.801161,-104.398839
4,2021,05,12019000,Porotos de soja excluidos p/siembra,35,8,3.508849e+08,648509571,541.063518,646.80,-105.736482,567.330362,-110.869638
5,2021,06,12019000,Porotos de soja excluidos p/siembra,40,7,3.238455e+08,614323340,527.158106,614.68,-87.521894,581.633740,-96.566260
6,2021,07,12019000,Porotos de soja excluidos p/siembra,30,7,2.546205e+08,501371528,507.848029,600.44,-92.591971,573.616903,-104.583097
7,2021,08,12019000,Porotos de soja excluidos p/siembra,26,7,2.020526e+08,396534230,509.546349,585.80,-76.253651,589.918630,-88.281370
8,2021,09,12019000,Porotos de soja excluidos p/siembra,13,4,1.311746e+08,251059402,522.484147,557.55,-35.065853,635.546136,-42.653864
9,2021,10,12019000,Porotos de soja excluidos p/siembra,16,5,1.362494e+08,262795989,518.460704,551.95,-33.489296,637.050547,-41.149453


### Tabla 1
Los valores son a precios capitalizados

In [32]:
contar_paises=df_producto.groupby('pais_descri').size().sort_values(ascending=False)
paises_interesantes=pd.DataFrame(contar_paises).reset_index().pais_descri.unique()[:4]
paises_interesantes

array(['Paraguay', 'Uruguay', 'Bolivia', 'Brasil'], dtype=object)

In [33]:
def datos_tabla_paises(pais):
    return [len(df_producto[df_producto.pais_descri==pais]),round(df_producto[df_producto.pais_descri==pais].fob_unitario_ton_capit.min(),1), round(df_producto[df_producto.pais_descri==pais].fob_unitario_ton_capit.quantile(0.25),1),round(df_producto[df_producto.pais_descri==pais].fob_unitario_ton_capit.quantile(0.5),1),round(df_producto[df_producto.pais_descri==pais].fob_unitario_ton_capit.quantile(0.75),1),round(df_producto[df_producto.pais_descri==pais].fob_unitario_ton_capit.min(),1)]

# A precios capitalizados
tabla_1_impo=pd.DataFrame({
    '':['No. of trade records','Lowest price','Lower quartile price','Median price','Upper quartile price','Highest price'],
    'Mundo':[len(df_producto),round(df_producto.fob_unitario_ton_capit.min(),1), round(df_producto.fob_unitario_ton_capit.quantile(0.25),1),round(df_producto.fob_unitario_ton_capit.quantile(0.5),1),round(df_producto.fob_unitario_ton_capit.quantile(0.75),1),round(df_producto.fob_unitario_ton_capit.min(),1)],
    paises_interesantes[0]:datos_tabla_paises(paises_interesantes[0]),
    paises_interesantes[1]:datos_tabla_paises(paises_interesantes[1]),
    paises_interesantes[2]:datos_tabla_paises(paises_interesantes[2]),
    paises_interesantes[3]:datos_tabla_paises(paises_interesantes[3])
})

tabla_1_impo=tabla_1_impo.transpose().reset_index()
header=tabla_1_impo.iloc[0]
tabla_1_impo=tabla_1_impo[1:]
tabla_1_impo.columns=header
tabla_1_impo['No. of trade records']=tabla_1_impo['No. of trade records'].astype(int)
tabla_1_impo

,,No. of trade records,Lowest price,Lower quartile price,Median price,Upper quartile price,Highest price
1,Mundo,448,366.9,462.6,478.5,497.1,366.9
2,Paraguay,347,418.2,465.1,479.7,501.6,418.2
3,Uruguay,46,442.6,472.7,481.9,492.1,442.6
4,Bolivia,30,378.7,436.7,456.5,470.7,378.7
5,Brasil,23,440.3,455.8,477.1,491.3,440.3


In [34]:
writer = pd.ExcelWriter(f'../output/importaciones soja.xlsx', engine='xlsxwriter')
tabla_1_impo.to_excel(writer, sheet_name='tabla_1_impo', index=False)
df_mensual_refer.to_excel(writer, sheet_name='agrupado mensual', index=False)
df_producto.to_excel(writer, sheet_name='desagregado', index=False)
writer.save()